<!--NAVIGATION-->
< | [Main Contents](https://vectorbite.github.io/VBiTraining2/) | >

# Mapping Climate Data to VecDyn 
##### Author: Deraj Wilson-Aggarwal, Imperial College London<br> deraj.wilson-aggarwal18@imperial.ac.uk <br> June 2019

There are a plethora of different ways to download climate data. When downloading climate data, it is important to choose a database that has the best coverage possible for your given task as they will differ in their spatial and temporal resolution. 

Three main ways to consider are:

- <b> Using pre-built Application Program Interfaces (APIs) such as RNOAA and RNCEP </b>
    
    APIs such as RNOAA and RNCEP are built packages for software such as R and Python that allows a user to extract climate data with ease. These packages call to servers to download the desired data. This method is great for smaller datasets, being reliable, convenient and efficient.
    Although they are convenient, APIs may be difficult to use without steep learning curves. Documentation may not always be widely available. For larger datasets, connecting to servers drastically increases the computational time required to download the relevant data so it is not recommended. <br><br>
    
- <b> Downloading open source data files and extracting required data from them </b>

    Downloading and extracting data manually from existing datasets generally requires a large amount of space and can seem clunky. However, when working on large datasets with a medium temporal resolution, they can be extremely efficient. The NetCDF file type is made specifically for climate data and are well suited for their purpose. For more info, please see https://www.unidata.ucar.edu/software/netcdf/docs/netcdf_introduction.html 
    <br><br>The main drawback of this method, is that it relies on external databases, as before, whilst also providing mid to low level temporal resolution. <br><br>

- <b> Obtaining raster files from satellite data and extracting data using High Performance Computing (HPC) </b>

    This is a method employed in GIS data analysis. Raster files can be huge, however have the ability to provide high temporal resolution in scales up to 5m. This makes them extremely powerful tools. Generally, the size of these files and the amount of datapoints requried, means that HPC methods are the most efficient ways of extracting the relevant data. <br><br>
    
    
This notebook will demonstrate how to map climate data to VecDyn data using open source NetCDF NOAA files. The example used is a subset of VecDyn and spans only 2 years, so will be relativlely small in size. I recommend to run any larger datasets using R or RStudio. Feedback on this notebook and its methodology are encouraged, so please do share alternative methods and suggestions with me. 




First step, let's download the data we need.  

You may download publicly available NOAA datasets following this link: ftp://ftp.cdc.noaa.gov/Datasets
My suggested datasets, and the ones I have chose to use are are:
- ftp://ftp.cdc.noaa.gov/Datasets/cpc_global_temp/ for daily maximum temperature data (degrees celsius) 
- ftp://ftp.cdc.noaa.gov/Datasets/cpc_global_precip/ for daily Precipitation 

These datasets are available as yearly files, with a spatial resolution of 0.5 degrees latitude and longitude (intervals are at .25 and .75). 

Unfortunately, the NOAA file co-ordinates are mapped slightly differently to that of VecDyn, so we need to match these together. The Climate Data Operators (CDO) can allow us to do this, so you will need to download the CDO application (https://code.mpimet.mpg.de/projects/cdo) and run the following code on the command line (from terminal):

``` cdo -z zip sellonlatbox,-180,180,-90,90 INPUT_FILE.nc OUTPUT_FILE.nc  ``` 

The following code will run through R to download the climate data you will require for this example. 

   <b> Please ensure you have the CDO application downloaded so that you can run it from the command line. <br><br> These files may take 5 minutes to download </b>

In [ ]:
# This download will take a couple of minutes as the files are about 60mb each! 

Years_to_download = c(2015,2016)

for (year in Years_to_download){
    
    # download the files that we require
    download.file(paste("ftp://ftp.cdc.noaa.gov/Datasets/cpc_global_precip/precip.",year,".nc", sep=""), paste("../data/precip.",year,".nc",sep=""))
    download.file(paste("ftp://ftp.cdc.noaa.gov/Datasets/cpc_global_temp/tmax.",year,".nc", sep=""), paste("../data/tmax.",year,".nc",sep=""))
    
    # Use R to go to the terminal and call the cdo remapping 
    system(paste("cdo -z zip sellonlatbox,-180,180,-90,90 ../data/precip.", year, ".nc ../data/precip.", year, ".new.nc", sep=""))
    system(paste("cdo -z zip sellonlatbox,-180,180,-90,90 ../data/tmax.", year, ".nc ../data/tmax.", year, ".new.nc", sep=""))
    
}

Next, let's load in our VecDyn data that we need climate data to map to. 

In [ ]:
#install.packages(ncdf4)
#install.packages(chron)
#install.packages(tidyr)

library(ncdf4)
library(chron)
library(tidyr)


# read in the data file without climate data 
Data_no_climate <- read.csv("../data/example_import_data.csv")

Data_climate <- Data_no_climate

# To keep things organised, let's rename the columns 
names(Data_climate)[35] <- "Initial_latitude"
names(Data_climate)[36] <- "Initial_longitude"
names(Data_climate)[28] <- "Collection.date.range"

##
## Please note that the indexes above may be a column out 
## due to including row names column when writing the csv 
## 
## If this doesnt work, please try the following code instead and re run:
#
#names(Data_climate)[34] <- "Initial_latitude"
#names(Data_climate)[35] <- "Initial_longitude"
#names(Data_climate)[27] <- "Collection.date.range"


Now we need to write a couple of functions to incorporate into our loop we use to populate the climate data columns. Some of the co-ordinates used are near to the coast - therefore it is possible that the we have ```NA``` inputs where the co-ordinate references the sea. 

Within the loop, we will include a number of "safety" statements in the form of if/else statements. Therefore, we will be able to catch any ```NA```s in our data frame and find the closest values we have. 

In [ ]:
# A function to round our co-ordinates to the nearest .25 value 
rounding <- function(value){
  # round to the nearest integer
  x <- round(value, digits = 0)
  # if rounded up (x>value) reduce by 0.25
  # else add 0.25 
  new = ifelse(x>value, x-0.25, x+0.25)
  return(new)
}

# A function to round to the other nearest 0.25 
rounding_opposite <- function(value){
  # round to the nearest integer
  x <- round(value, digits = 0)
  # if rounded up (x>value) reduce by 0.25
  # else add 0.25 
  new = ifelse(x>value, x+0.25, x-0.25)
  return(new)
}

# A function to force rounding up if required 
force_round_up <- function(value){
  # round UP the nearest integer 
  # minus 0.25 to back towards value 
  x <- ceiling(value) - 0.25
  return(x)
}


Now we have defined some functions to round, we can apply them to our current co-ordinate columns. 

In [ ]:
# Round the latitude and longitude using our previous functions
# The new co-ordinates are saved as new columns 
Data_climate$latitude <- rounding(as.numeric(Data_climate$Initial_latitude))
Data_climate$longitude <- rounding(as.numeric(Data_climate$Initial_longitude))


We also need to ensure that the columns we use are correctly formatted.

In [ ]:
# convert the date 
Data_climate$Collection.date.range <- as.Date(Data_climate$Collection.date.range)

# create a column for the Julian date for later slicing  
Data_climate$Julian <- as.numeric(format(Data_climate$Collection.date.range, "%j"))

# extract year fro mthe date 
Data_climate$Year <- as.numeric(format(Data_climate$Collection.date.range, "%Y"))

# order by date and reset indexes 
Data_climate <- Data_climate[order(Data_climate$Collection.date.range, decreasing = FALSE),]
rownames(Data_climate) <- NULL

# Create a column of NAs to populate 
Data_climate$Precipitation <- NA

# Create a column of NAs to populate 
Data_climate$Max.Temp <- NA



Now this is where it might get a little messy and difficult to follow - so bear with me and read through the comments in the loop below one by one. The code is (currently) rather verbose, and is in the process of being optimised. All suggestions are very welcome!

The process, in words, is as follows: 

- For each unique year, find the row indexes that apply to that year
- Open the NetCDF files for that year relating to Precipitation and Maximum Temperature 
- Loop through each row index and extract the lat/long
- Extract the precipitation and temperature value for the lat/long
- If there is an ```NA``` then try again with different lat/long combinations 

In [ ]:
i <- 0
Year1 <- 0

# for each unique year in the dataframe 
for (Year in unique(Data_climate$Year)){
  print(paste("Adding climate data for the year ", Year, sep=""))
  # Draw out the index values in the dataset that correspond to that year 
  Year_Index <- which(Data_climate[,"Year"] == Year)
  
  ###############################################
  # Extract daily precipitation 
  ###############################################
  
  print("Opening NetCDF precipitation file")
  # Define the netCDF file for that year stored in the data directory (please ask for this data as it is very large)
  ncpath_prp <- "../data/"
  ncfname_prp <- paste(ncpath_prp, "precip.", Year, ".new", ".nc", sep="")
  dname_prp <- "precip" 
  
  # open the netCDF file
  ncin_prp <- nc_open(ncfname_prp)
  
  # get longitude and latitude for the factor 
  lon_prp <- ncvar_get(ncin_prp,"lon")
  lat_prp <- ncvar_get(ncin_prp,"lat")
  
  # get the time for that factor 
  time <- ncvar_get(ncin_prp,"time")
  tunits <- ncatt_get(ncin_prp,"time","units")
  nt <- dim(time)
  
  # Create an array for the precipitation values 
  prp_array <- ncvar_get(ncin_prp,dname_prp)
  
  # other factors that may be required/useful
  dlname <- ncatt_get(ncin_prp,dname_prp,"long_name")
  dunits <- ncatt_get(ncin_prp,dname_prp,"units")
  fillvalue <- ncatt_get(ncin_prp,dname_prp,"_FillValue")
  
  # replace netCDF fill values with NA's
  prp_array[prp_array==fillvalue$value] <- NA
  
  # create a matrix for the lon/lats of that file 
  lonlat_prp <- as.matrix(expand.grid(lon_prp,lat_prp))
  
  ###############################################
  # Extract max daily temperature
  ###############################################
  
  print("Opening NetCDF temperature file")
  ncpath_tmp <- "../data/"
  ncfname_tmp <- paste(ncpath_tmp, "tmax.", Year, ".new", ".nc", sep="")
  dname <- "tmax" 
  
  
  # open the netCDF file
  ncin_tmp <- nc_open(ncfname_tmp)
  
  # get longitude and latitude for factor 
  lon_tmp <- ncvar_get(ncin_tmp,"lon")
  lat_tmp <- ncvar_get(ncin_tmp,"lat")
  
  # extract time
  time <- ncvar_get(ncin_tmp,"time")
  tunits <- ncatt_get(ncin_tmp,"time","units")
  nt <- dim(time)
  
  # get temperature 
  tmp_array <- ncvar_get(ncin_tmp,dname)
  
  dlname <- ncatt_get(ncin_tmp,dname,"long_name")
  dunits <- ncatt_get(ncin_tmp,dname,"units")
  fillvalue <- ncatt_get(ncin_tmp,dname,"_FillValue")
  
  # replace netCDF fill values with NA's
  tmp_array[tmp_array==fillvalue$value] <- NA
  
  # define a matric with temperature lon/lats
  lonlat_tmp <- as.matrix(expand.grid(lon_tmp,lat_tmp))
  print(paste("Running for the year ", Year,". There are ", length(Year_Index), " rows, from ", min(Year_Index), "to", max(Year_Index)))
  
  # loop to extract climate data and save
  for (row in Year_Index){
    # define the Julian day for that row 
    j <- Data_climate[row, "Julian"]
    # define the lon/lats of the row 
    lon_n <- Data_climate[row, "longitude"]
    lat_n <- Data_climate[row, "latitude"]
    
    # If the julian day differs, or the year differs from the previous row
    if (j!=i | Year != Year1){
      # redefine i as j 
      i <- j
      # Slice the temp data into the day required
      tmp_slice <- tmp_array[,,i]
      # store as a vector
      tmp_vec <- as.vector(tmp_slice)
      # bind as a dataframe with the lon/lats of the factor 
      tmp_df01 <- data.frame(cbind(lonlat_tmp,tmp_vec))
      # add the temp value for the rows long/lats on tht day  
      Data_climate[row, "Max.Temp"] <- with(tmp_df01, tmp_vec[Var2 == lat_n & Var1 == lon_n])
      
      # Same again but for precipitation 
      prp_slice <- prp_array[,,i]
      prp_vec <- as.vector(prp_slice)
      prp_df01 <- data.frame(cbind(lonlat_prp,prp_vec))
      Data_climate[row, "Precipitation"] <- with(prp_df01, prp_vec[Var2 == lat_n & Var1 == lon_n])
      
    }
    # otherwise, if Julian day does not differ NOR does the year 
    else{
      # Go straight in and extract from the existing slice for that lon/lat
      Data_climate[row, "Max.Temp"] <- with(tmp_df01, tmp_vec[Var2 == lat_n & Var1 == lon_n])
      Data_climate[row, "Precipitation"] <- with(prp_df01, prp_vec[Var2 == lat_n & Var1 == lon_n])
      
    }
    # redefine the previous year once the years index has been completed
    
    if (is.na(Data_climate[row, "Max.Temp"])){
      # Try a different latitude
      Data_climate[row, "latitude"] <- rounding(Data_climate[row, "Initial_latitude"])
      Data_climate[row, "longitude"] <- rounding_opposite(Data_climate[row, "Initial_longitude"])
      
      # define the lon/lats of the row
      lon_n <- Data_climate[row, "longitude"]
      lat_n <- Data_climate[row, "latitude"]
      #print(paste("Attempting again for row ", row, "at lon/lat ", lon_n, ",",lat_n))
      
      
      # add the temp value for the rows long/lats on tht day
      Data_climate[row, "Max.Temp"] <- with(tmp_df01, tmp_vec[Var2 == lat_n & Var1 == lon_n])
      
      
      if (is.na(Data_climate[row,"Max.Temp"])){
        # try a different longitude
        Data_climate[row, "latitude"] <- rounding_opposite(Data_climate[row, "Initial_latitude"])
        Data_climate[row, "longitude"] <- rounding(Data_climate[row, "Initial_longitude"])
        
        # define the lon/lats of the row
        lon_n <- Data_climate[row, "longitude"]
        lat_n <- Data_climate[row, "latitude"]
        #print(paste("Attempting second time for row ", row, "at lon/lat ", lon_n, ",", lat_n))
        
        Data_climate[row, "Max.Temp"] <- with(tmp_df01, tmp_vec[Var2 == lat_n & Var1 == lon_n])
        
        if (is.na(Data_climate[row,"Max.Temp"])){
          # try a different longitude
          Data_climate[row, "latitude"] <- rounding_opposite(Data_climate[row, "Initial_latitude"])
          Data_climate[row, "longitude"] <- rounding_opposite(Data_climate[row, "Initial_longitude"])
          
          # define the lon/lats of the row
          lon_n <- Data_climate[row, "longitude"]
          lat_n <- Data_climate[row, "latitude"]
          print(paste("Attempting third time for row ", row, "at lon/lat ",lon_n,",",lat_n))
          
          Data_climate[row, "Max.Temp"] <- with(tmp_df01, tmp_vec[Var2 == lat_n & Var1 == lon_n])
          
          if (is.na(Data_climate[row,"Max.Temp"])){
            # Try a different latitude
            Data_climate[row, "latitude"] <- rounding(Data_climate[row, "Initial_latitude"])
            Data_climate[row, "longitude"] <- force_round_up(Data_climate[row, "Initial_longitude"])
            
            # define the lon/lats of the row
            lon_n <- Data_climate[row, "longitude"]
            lat_n <- Data_climate[row, "latitude"]
            #print(paste("Attempting again for row ", row, "at lon/lat ", lon_n, ",",lat_n))
            
            
            # add the temp value for the rows long/lats on tht day
            Data_climate[row, "Max.Temp"] <- with(tmp_df01, tmp_vec[Var2 == lat_n & Var1 == lon_n])
            
            if (is.na(Data_climate[row, "Max.Temp"])){
              
              Data_climate[row, "latitude"] <- force_round_up(Data_climate[row, "Initial_latitude"])
              Data_climate[row, "longitude"] <- rounding(Data_climate[row, "Initial_longitude"])
              
              # define the lon/lats of the row
              lon_n <- Data_climate[row, "longitude"]
              lat_n <- Data_climate[row, "latitude"]
              #print(paste("Attempting second time for row ", row, "at lon/lat ", lon_n, ",",lat_n))
              
              
              # add the temp value for the rows long/lats on tht day
              Data_climate[row, "Max.Temp"] <- with(tmp_df01, tmp_vec[Var2 == lat_n & Var1 == lon_n])
            }
          }
        }
      }
    }
    
    Year1 == Year
  } 
  
}



It's important to check the values that we've obtained are as we expect using ```summary()```.

In [ ]:
summary(Data_climate$Precipitation)
summary(Data_climate$Max.Temp)

Finally, let's save our output to a new ```.csv``` file, ready to import and analyse:


In [ ]:
# write.csv(Data_climate, "../data/example_import_data_with_climate.csv", row.names=FALSE)